

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_GM_DE.ipynb)




# **ICD10 coding for German**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

Import license keys

In [ ]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP Version: 2.6.0
SparkNLP-JSL Version: 2.6.0


Install dependencies

In [ ]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 60kB/s 
     |████████████████████████████████| 204kB 34.7MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.6.0-8388813d58b67fa25bf9cf603393363af96dba16


Import dependencies into Python

In [ ]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


Start the Spark session

In [ ]:
spark = sparknlp_jsl.start(secret)

## 2. Construct the pipeline

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer()\
    .setInputCols(['sentences']) \
    .setOutputCol('tokens')

embeddings = WordEmbeddingsModel.pretrained('w2v_cc_300d','de', 'clinical/models') \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

ner_model = NerDLModel.pretrained('ner_healthcare','de', 'clinical/models') \
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")   

#Filtering NER chunks as ICD codes only relate to specific entities.
ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunk").setWhiteList(['MEDICAL_CONDITION'])

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("ner_chunk", "embeddings")\
    .setOutputCol("chunk_embeddings")

entity_resolver = ChunkEntityResolverModel\
    .pretrained("chunkresolve_ICD10GM","de","clinical/models")\
    .setInputCols("tokens","chunk_embeddings").setOutputCol("resolution")
 
    
pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    embeddings,
    ner_model,
    ner_chunker,
    chunk_embeddings,
    entity_resolver])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = pipeline.fit(empty_df)

light_pipeline = sparknlp.base.LightPipeline(pipeline_model)

w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_healthcare download started this may take some time.
Approximate size to download 14.1 MB
[OK!]
chunkresolve_ICD10GM download started this may take some time.
Approximate size to download 14.8 MB
[OK!]


## 3. Create example inputs

In [ ]:
# Enter examples as strings in this array
input_list = [
     """Nach der Einnahme von Symbioflor2 (1992) entwickelte sich dann dieser Fäkalgeruch bei mir, den ich seit dem nicht mehr losgeworden bin. Vom Symbioflor2 bekam ich dann auch immer stärkere körperliche Beschwerden, so daß ich es mit der Angst zu tun bekam und meinen Arzt bat mir Cotrim zuverschreiben (das zweitemal nach 1983). Die Folge war leider, daß ich starken Fußschweiß bekam mit entsprechendem Geruch. Auch meine Hände fingen stark an zu riechen.
Ich war im Schlaflabor. Nur eine Auffälligkeit. In der ersten Nacht dort trat ich verfrüht in den Traumschlaf ein, was ein Kriterium für Narkolepsie ist. Bluttest und Gentest aber waren negativ. Meine Lungen sind super, mein MRT unauffällig. Jetzt bin ich in einer neurologischen Spezialklinik. Der Arzt meinte, dass eine sehr geringe Wahrscheinlichkeit besteht, dass ich doch an Narkolepsie leide. Ich soll in einem halben Jahr nochmal dort ins Schlaflabor. Medikamente gegen die Müdigkeit will er mir nicht verschreiben. Antidepressiva wären es, aber die müsste ich mir abgewöhnen, bevor ich in das Schlaflabor gehe. Also lieber "clean" bleiben.
Ich glaube mittlerweile, dass das Problem ganz woanders liegt... obwohl ich als Kind laut Aussagen meiner Eltern immer recht schnell dabei war, was laufen, Fahrradfahren etc. angeht, habe ich jetzt gerade als Erwachsener gemerkt, dass ich sehr langsam dabei bin, komplexe Bewegungen einzulernen und richtig auszuführen, sei es z.B. Tanzen oder Kampfsport. Wobei das sicherlich was ist, was mit der Zeit und Training schon deutlich besser geworden ist.""",
]

# 4. Run the pipeline

In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)
light_result = light_pipeline.fullAnnotate(input_list[0])

# 5. Visualize

Full Pipeline

In [ ]:
result.select(
    F.explode(
        F.arrays_zip('ner_chunk.result', 
                     'ner_chunk.begin',
                     'ner_chunk.end',
                     'ner_chunk.metadata',
                     'resolution.metadata', 'resolution.result')
    ).alias('cols')
).select(
    F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']").alias('begin'),
    F.expr("cols['2']").alias('end'),
    F.expr("cols['3']['entity']").alias('entity'),
    F.expr("cols['4']['resolved_text']").alias('icd10_description'),
    F.expr("cols['5']").alias('icd10_code'),
).toPandas()

,chunk,begin,end,entity,icd10_description,icd10_code
0,Beschwerden,199,209,MEDICAL_CONDITION,Acné excoriée Akne,L70.5
1,Angst,234,238,MEDICAL_CONDITION,Hautpest Pest,A20.1
2,Auffälligkeit,486,498,MEDICAL_CONDITION,Sonstige abnorme Befunde Blutchemie Sonstige a...,R79
3,Narkolepsie,590,600,MEDICAL_CONDITION,Narkolepsie Kataplexie Schlafstörungen,G47.4
4,Lungen,654,659,MEDICAL_CONDITION,Parakokzidioidomykose Lunge Parakokzidioidomykose,B41.0
5,leide,845,849,MEDICAL_CONDITION,Acné excoriée Akne,L70.5
6,Müdigkeit,934,942,MEDICAL_CONDITION,Unwohlsein Ermüdung Unwohlsein Ermüdung,R53


Light Pipeline

In [ ]:
light_result[0]['resolution']

[Annotation(entity, 199, 209, L70.5, {'chunk': '0', 'all_k_results': 'L70.5:::R05:::L70:::L70.8:::R45.4', 'all_k_sorensen_distances': '1.0000:::1.0000:::1.0000:::1.0000:::1.0000', 'all_k_levenshtein_distances': '0.7778:::0.7692:::0.9091:::0.7778:::0.8372', 'all_k_distances': '2.0000:::2.0000:::2.5515:::2.6679:::3.5310', 'all_k_jaro_distances': '0.4300:::0.5200:::0.5800:::0.4900:::0.5000', 'all_k_jaccard_distances': '1.0000:::1.0000:::1.0000:::1.0000:::1.0000', 'confidence': '0.3026', 'all_k_resolutions': 'Acné excoriée Akne:::Husten Husten:::Akne Akne:::Sonstige Akne Akne:::Reizbarkeit Wut Symptome Stimmung betreffen', 'target_text': 'Beschwerden', 'token': 'Beschwerden', 'resolved_text': 'Acné excoriée Akne', 'all_k_confidences': '0.3026:::0.3026:::0.1743:::0.1552:::0.0654', 'all_k_tfidf_distances': '-1.0000:::-1.0000:::-1.0000:::-1.0000:::-1.0000', 'all_k_wmd_distances': '1.5515:::1.8329:::2.5515:::2.6679:::3.5310', 'distance': '2.0000', 'sentence': '1'}),
 Annotation(entity, 234, 23